In [1]:
"""
DO NOT RUN

'Data Preparation' is only used to prepare datasets that are utilized in 'Data Visualization' & 'Data Observation'
"""

"""
import pandas as pd
import csv
import math

#Takes in a list of numbers and returns the mean of the list
def MeanCalc(dataList):
    mean = 0
    for i in dataList:
        mean += float(i)
    mean = mean/len(dataList)
    return mean

#Takes in a sorted list of numbers and returns the median of the list
def MedianCalc(dataList):
    count = 0
    if (len(dataList)%2) == 0:
        even = True
    else:
        even = False
    for i in dataList:
        count+=1
        if even:
            if count == (len(dataList)/2):
                median = (dataList[count-2] + i) / 2
        else:
            if count-(len(dataList)/2) > 0 and count-(len(dataList)/2) < 1:
                median = i
    return median

#creating a list of NYC zip codes
ZIPLIST = []
with open("ZipCodes.txt", 'r') as z:
    while True:
        content = z.readline()
        if not content:
            break
        content_list = content.split(' \t')
        del content_list[2]
        content_list[3] = content_list[3][:-1]
        ZIPLIST.append(content_list[0])
        ZIPLIST.append(content_list[2])

#Program to assist with manually creating a file of the zip codes contained within each NYC neighborhood        
borList = []
tempZip = []
with open("newMedianAskingRent.csv") as newMRent:
    MedRent = csv.reader(newMRent, delimiter=',')
    for line in MedRent:
        if line[2] == 'neighborhood':
            borList.append(line[0])
BORZIPLIST = {}
for bor in borList:
    tempZipstr = input("Zip Codes for " + bor + ": ")
    tempZip = tempZipstr.split(', ')
    BORZIPLIST[bor] = tempZip
    print("%s: %s" % (bor, BORZIPLIST[bor]))

#Write the zip code data to BoroughZipCodes.csv
with open("BoroughZipCodes.csv", "w") as BorZips:
    BorZips.write('Borough, Zip Codes\n')
    for key in BORZIPLIST.keys():
        tempWrite = ''
        first = True
        for i in BORZIPLIST[key]:
            if first:
                tempWrite += '%s'%(i)
                first = False
            else:
                tempWrite += ',%s'%(i)
        BorZips.write('%s,%s\n' % (key, tempWrite))  

ZInfoDict = {}
#List containing the future list of values under each neighborhood in "NInfoDict"
DictKey = ["Neighborhood", "Median Income", "Mean Income", "Median Margin of Error", "Mean Margin of Error", "Median Rent Price", "30% Income Rent", "30% Income Rent Margin of Error", "Actual Rent vs. 30% Income Rent"]
count = 0
first = True

#keeps zipcodes, median/median margin of error, and mean/mean margin of error; also removes empty or invalid data points
removedZips = []
with open("newAvgIncomeData.csv") as newAInc:
    AvgIncData = csv.reader(newAInc, delimiter=',')
    for line in AvgIncData:
        if first:
            first = False
        else:
            zipcode = line[0][0:5]
            if count >= 8: #when the line reaches the first line of a new zip code, reset the count
                count = 0
            if count == 0: #first line of a zipcode contains median and mean data
                median = line[12][2:-1]
                mean = line[13][2:-1]
                if median == '-' or median == '**':
                    removedZips.append(zipcode)
                    continue
                else:
                    ZInfoDict[zipcode] = [median, mean]
                count += 1
            elif count == 1: #second line of a zipcode contains margins of error
                medianMOE = line[12][2:-1]
                meanMOE = line[13][2:-1]
                if medianMOE == '***':
                    del ZInfoDict[zipcode]
                    removedZips.append(zipcode)
                else:
                    ZInfoDict[zipcode].append(medianMOE)
                    ZInfoDict[zipcode].append(meanMOE)
                count += 1
            else:
                count += 1

#Consolidates Zipcode based income data into neighborhood income data by taking the average of all zip codes in each neighborhood           
NInfoDict = {}
with open("BoroughZipCodes.csv") as BZipCodes:
    first = True
    BZips = csv.reader(BZipCodes, delimiter=',')
    for line in BZips:
        tempAvgA = 0.0 #Temp avg of the mean
        tempAvgM = 0.0 #Temp avg of the median
        tempAMOE = 0.0 #Temp margin of error for mean
        tempMMOE = 0.0 #Temp margin of error for median
        tempData = []
        count = 0
        if first:
            first = False
            continue
        NInfoDict[line[0]] = []
        if len(line) == 2:
            NInfoDict[line[0]] = ZInfoDict[line[1]]
            continue
        first = True
        for code in line:
            if first:
                first = False
                continue
            if code in removedZips:
                continue
            else:
                count += 1
                ZipData = ZInfoDict[code]
                tempAvgM += float(ZipData[0])
                tempAvgA += float(ZipData[1])
                tempMMOE += (float(ZipData[2][1:]))**2
                tempAMOE += (float(ZipData[3][1:]))**2
        tempData.append(tempAvgM / count)
        tempData.append(tempAvgA / count)
        tempData.append('±%s' %(str(math.sqrt(tempMMOE / count))))
        tempData.append('±%s' %(str(math.sqrt(tempAMOE / count))))
        NInfoDict[line[0]] = tempData

#Used solely for testing
#Consolidating all rent monthly data into a single total estimated rent over the past 5 years
first = True
with open('newMedianAskingRent.csv') as MedAskRent:
    MedRent = csv.reader(MedAskRent, delimiter=',')
    for line in MedRent:
        begin = 0
        if first:
            first = False
        else:
            if line[2] != "neighborhood":
                continue
            borough = line[0]
            count = 0
            total = 0
            for col in line:
                if begin < 3:
                    begin += 1
                else:
                    if col == '':
                        continue
                    total += float(col)
                    count += 1
            NInfoDict[borough].append(total / count)

#Used solely for testing
#Calculates the 30% income threshold for maximum recommended rent and the difference between that rent and the actual median rent
for key in NInfoDict:
    income = float(NInfoDict[key][0])
    rent = float(NInfoDict[key][4])
    MoE = float(NInfoDict[key][2][1:])
    oRentMoE = (MoE/12) * 0.3
    oRent = (income/12) * 0.3
    NInfoDict[key].append(oRent)
    NInfoDict[key].append('±' + str(oRentMoE))
    dRent = rent - oRent
    NInfoDict[key].append(dRent)

#Used solely for testing
#Writes all relevant data into a single .csv file
with open('NYCIncomeRentData.csv', 'w') as FinalData:
    first = True
    for i in DictKey:
        if first:
            FinalData.write(i)
            first = False
            continue
        FinalData.write(',' + i)
    for key in NInfoDict:
        FinalData.write('\n' + key + ',')
        first = True
        for item in NInfoDict[key]:
            if first:
                FinalData.write(str(item))
                first = False
                continue
            FinalData.write(',' + str(item))

#Gathering quarterly rent data from rent file and writing it to QuarterlyRentData.csv
newMData = pd.read_csv('newMedianAskingRent.csv')
QDict = {'Q18.2': [], 'Q18.3': [], 'Q18.4': [], 'Q19.1': [], 'Q19.2': [], 'Q19.3': [], 'Q19.4': [], 'Q20.1': [], 'Q20.2': [], 'Q20.3': [], 'Q20.4': [], 'Q21.1': [], 'Q21.2': [], 'Q21.3': [], 'Q21.4': [], 'Q22.1': [], 'Q22.2': [], 'Q22.3': [], 'Q22.4': []}
QFDict = {'2018': [], '2019': [], '2020': [], '2021': [], '2022': []}
tempIndex = []
for index, row in newMData.iterrows():
    if row[2] == "submarket" or row[2] == "borough":
        continue
    count = 3
    YQcount = 2
    Qcount = 1
    for i in row[3:-4]:
        if Qcount > 3:
            Qcount = 1
            YQcount+=1
        if YQcount > 4:
            YQcount = 1
        if math.isnan(i):
            pass
        else:
            QDict['Q'+str(newMData.columns.values[count])[2:4]+'.'+str(YQcount)].append(i)
        Qcount+=1
        count+=1
first = True
for i in QDict.keys():
    QDict[i] = MedianCalc(QDict[i])
    if first:
        first = False
        QFDict['2018'].append('')
    QFDict['20' + i[1:3]].append(QDict[i])
tempDF = pd.DataFrame(data=QFDict.values(), index=QFDict.keys(), columns=['Q1', 'Q2', 'Q3', 'Q4'])
tempDF.to_csv('QuarterlyRentData.csv')

#Getting monthly rent data during/post Covid(Jan 2020 and beyond) and writing it to CovidRentData.csv
colMean = []
beforeData = True
colNames = []
for i in newMData.columns:
    if i == '2020-01':
        beforeData = False
    if beforeData:
        continue
    colNames.append(i)
for col in colNames:
    count = 0
    tempList = []
    for num in newMData[col]:
        if math.isnan(num):
            continue
        tempList.append(num)
    tempList.sort()
    colMean.append(MeanCalc(tempList))
csvCovidDict = {'Year-Month': colNames, 'Mean Rent': colMean}
tempDF = pd.DataFrame(csvCovidDict)
tempRows = []
tempDF.to_csv('CovidRentData.csv')

#Getting rent data divided by Boroughs and writing it to BoroughRentData.csv
BorRows = {'Bronx': [], 'Brooklyn': [], 'Manhattan': [], 'Queens': []}
areaType = newMData['areaType']
for i in range(len(areaType)):
    row1 = newMData.iloc(0)[i]
    if areaType[i] == 'borough':
        if row1[0] == 'Staten Island':
            continue
        count = 0
        tempRows = []
        for i in row1[24:-4]:#starts at 2020, ends at start of 2023
            if count > 11:
                count = 0
                BorRows[row1[0]].append(MedianCalc(tempRows))
                tempRows = []
            tempRows.append(i)
            count+=1
        BorRows[row1[0]].append(MedianCalc(tempRows))
tempDF = pd.DataFrame(BorRows.values(), BorRows.keys(), ['2020', '2021', '2022'])
tempDF.to_csv('BoroughRentData.csv')

#Gathering 2021 median rent
rentDict = {}
for index, row in newMData.iterrows():
    if row[2] == "submarket" or row[2] == "borough":
        continue
    currRow = row[0]
    rentDict[currRow] = []
    for i in row[36:48]:
        if math.isnan(i):
            continue
        rentDict[currRow].append(i)
    rentDict[currRow].sort()
    rentDict[currRow] = MedianCalc(rentDict[currRow])

#Coalescing and writing 2021 rent, income, and rent % data to MedianRentIncData.csv
incDict = {}
with open('NYCIncomeRentData.csv') as nycIncData:
    fullData = csv.reader(nycIncData, delimiter=',')
    first = True
    for line in fullData:
        currLine = line[0]
        if first:
            first = False
            continue
        incDict[currLine] = [float(line[1])/12]
        incDict[currLine].append(rentDict[currLine])
        incDict[currLine].append((incDict[currLine][1]/incDict[currLine][0]) * 100)
tempDF = pd.DataFrame(incDict.values(), incDict.keys(), ['Median Monthly 2021 Income', 'Median 2021 Rent', 'Actual % of Income Spent on Rent 2021'])
tempDF.to_csv('MedianRentIncData.csv')
"""  